# Training the Food101 Model with EfficientNetB2 🍔

## 📚 Import Libraries

In [13]:
import sys
import torchvision
import torch

## 🛠️ Create EfficientNetB2 Model

In [ ]:
import foodgenius.model_builder as model_builder

effnetb2_101, effnetb2_transforms = model_builder.create_effnetb2_model(num_classes=101)

food_101_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomRotation(15),  
    torchvision.transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  
    torchvision.transforms.RandomHorizontalFlip(),  
    torchvision.transforms.ColorJitter(contrast=0.1), 
    torchvision.transforms.TrivialAugmentWide(), 
    effnetb2_transforms,
                                                    ])

food_101_transforms

## 📦 Load Food101 Dataset

In [21]:
from torchvision import datasets

from pathlib import Path

data_dir = Path("./data")

train_data = datasets.Food101(root=data_dir, split="train", transform=food_101_transforms, download=True)
test_data = datasets.Food101(root=data_dir, split="test", transform=food_101_transforms, download=True)


food101_class_names = train_data.classes

## ✂️ Split the Dataset

In [22]:
import foodgenius.utils as utils

train_data, _ = utils.split_dataset(train_data, split_size=0.2)
test_data, _ = utils.split_dataset(test_data, split_size=0.2)

## 📊 Create DataLoaders

In [23]:
import torch

BATCH_SIZE=32

train_dataloader_food101 = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_dataloader_food101 = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

## 🏋️ Train the Model

In [ ]:
import foodgenius.engine as engine

optimizer = torch.optim.Adam(effnetb2_101.parameters(), lr=0.001)

loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)


effnetb2_101_results = engine.train(model=effnetb2_101.to('cuda:0'),
                                    train_dataloader=train_dataloader_food101,
                                    test_dataloader=test_dataloader_food101,
                                    optimizer=optimizer,
                                    loss_fn=loss_fn,
                                    epochs=5,
                                    device=torch.device('cuda'))


## 💾 Save the Trained Model

In [ ]:
utils.save_model(model=effnetb2_101, target_dir="models", model_name='pretrained_effnetb2_feature_extractor_food_101.pth')

[INFO] Saving model to: models/09_pretrained_effnetb2_feature_extractor_food_101.pth
